<a href="https://colab.research.google.com/github/AhmedFarrukh/DeepLearning-EdgeComputing/blob/main/Reproducing_Paper_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In an effort to reproduce the findings of the paper, "To Compress, or Not to Compress: Characterizing Deep Learning Model Compression for Embedded Inference", 7 popular convolutional neural network models will be trained, quantized and then tested for accuracy and inference time.

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import sys
import time
import numpy as np
import pathlib

In [ ]:
modelNames = ["MobileNet", "ResNet50", "ResNet101", "InceptionV3", "VGG16", "VGG19", "ResNet152"]

In [ ]:
for modelName in modelNames:
  model = tf.keras.applications.MobileNet(
    weights='imagenet'
  )

  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tflite_model = converter.convert()

  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  tflite_model_quant = converter.convert()

  tflite_models_dir = pathlib.Path("/tmp/tflite_models/")
  tflite_models_dir.mkdir(exist_ok=True, parents=True)

  # Save the unquantized/float model:
  tflite_model_file = tflite_models_dir/(modelName+".tflite")
  tflite_model_file.write_bytes(tflite_model)
  # Save the quantized model:
  tflite_model_quant_file = tflite_models_dir/(modelName+"_quant.tflite")
  tflite_model_quant_file.write_bytes(tflite_model_quant)


17225924/17225924 [==============================] - 1s 0us/step


In [ ]:
!mkdir /tmp/benchmark
!wget https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/linux_x86-64_benchmark_model -P /tmp/benchmark
!chmod +x /tmp/benchmark/linux_x86-64_benchmark_model
!touch /tmp/benchmark/results

--2024-06-28 11:11:46--  https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/linux_x86-64_benchmark_model
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.207, 108.177.96.207, 108.177.119.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6237672 (5.9M) [application/octet-stream]
Saving to: ‘/tmp/benchmark/linux_x86-64_benchmark_model’

linux_x86-64_benchm 100%[===================>]   5.95M  8.01MB/s    in 0.7s    

2024-06-28 11:11:47 (8.01 MB/s) - ‘/tmp/benchmark/linux_x86-64_benchmark_model’ saved [6237672/6237672]



In [6]:
for modelName in modelNames:
  os.system("echo \"" + modelName + "; Original\n\" >> /tmp/benchmark/results" )

  os.system("/tmp/benchmark/linux_x86-64_benchmark_model \
    --graph=/tmp/tflite_models/" + modelName +".tflite"+" \
    --num_threads=1 >> /tmp/benchmark/results")

  os.system("echo \"\n" + modelName + "; Quantized\n\" >> /tmp/benchmark/results" )

  os.system("/tmp/benchmark/linux_x86-64_benchmark_model \
    --graph=/tmp/tflite_models/" + modelName +"_quant.tflite"+" \
    --num_threads=1 >> /tmp/benchmark/results")

  os.system("echo \"" + "\n\n\" >> /tmp/benchmark/results" )

f = open("/tmp/benchmark/results", "r")
print(f.read())


MobileNet; Original

INFO: STARTING!
INFO: Log parameter values verbosely: [0]
INFO: Num threads: [1]
INFO: Graph: [/tmp/tflite_models/MobileNet.tflite]
INFO: Signature to run: []
INFO: #threads used for CPU inference: [1]
INFO: Loaded model /tmp/tflite_models/MobileNet.tflite
INFO: The input model file size (MB): 16.9034
INFO: Initialized session in 130.151ms.
INFO: Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.
INFO: count=21 first=24494 curr=23378 min=23340 max=26421 avg=24077.7 std=750

INFO: Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.
INFO: count=50 first=23758 curr=23574 min=23221 max=36097 avg=24795.7 std=2277

INFO: Inference timings in us: Init: 130151, First inference: 24494, Warmup (avg): 24077.7, Inference (avg): 24795.7
INFO: Note: as the benchmark tool itself affects memory footprint, the following is only APPROXIMATE to the actual memory footprint